# Derive Wind Speed from the Direct Model Outputs of ICON-CH1-EPS

This notebook demonstrates the full workflow for accessing perturbed ICON-CH1-EPS wind forecast data, calculating wind speed and visualizing the results. The data is provided by MeteoSwiss as part of Switzerland’s [Open Government Data (OGD) initiative](https://www.meteoswiss.admin.ch/services-and-publications/service/open-data.html).

The core functionality is powered by the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — developed in-house to simplify working with numerical weather model data. It includes the `ogd_api` module for data access, along with utilities for applying temporal and spatial operators to model output, such as calculating wind speed and regridding.

For visualization, this notebook uses the [Matplotlib](https://matplotlib.org/stable/index.html) library, a well-known visualization tool in Python. 

---

## 🔍 **What You’ll Do in This Notebook**

 🛰️  **Retrieve**  
    Fetch perturbed ICON-CH1-EPS forecast data (e.g., 10m eastward and 10m northward wind) via the `ogd_api` module.

 📐  **Calculate**  
    Calculate horizontal wind speed using [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/)'s `wind` module and apply ensemble statistics..

 🧭  **Regrid**  
    Interpolate ICON-CH1-EPS data from its native, icosahedral grid to the regular [Swiss grid LV95+ EPSG:2056](https://epsg.io/2056).

 🌍  **Visualize**  
    Plot each ensemble member on a map with meteorological styling using [Matplotlib](https://matplotlib.org/stable/index.html).

---

## Retrieving Precipitation Forecast
In this first part, we retrieve perturbed eastward and northward wind forecast data from the numerical weather model ICON-CH1-EPS. To access this data, we use the `ogd_api` module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON forecast datasets interactively before writing code, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.


![browser-ch2-resized.PNG](./images/browser-ch2-resized.PNG)

### Creating Requests
To retrieve the forecast data, we first define API requests using the `ogd_api.Request` class. In this example, two requests are specified: one for accessing grid eastward wind 10 meter above the surface (`U_10M`) and the other for accessing grid northward wind 10 meter above the surface (`V_10M`). Each request targets perturbed data at the initialization time of the forecast.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [ ]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

# Set default date of today midnight in UTC
today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

# retrieve 10 meter eastward wind
req_U10 = ogd_api.Request(
    collection="ogd-forecasting-icon-ch1",
    variable="U_10M",
    reference_datetime=today_midnight_utc,
    perturbed=True,
    horizon="P0DT0H",
)

# retrieve 10 meter northward wind
req_V10 = ogd_api.Request(
    collection="ogd-forecasting-icon-ch1",
    variable="V_10M",
    reference_datetime=today_midnight_utc,
    perturbed=True,
    horizon="P0DT0H",
)

Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch1`). |
| `variable`           | Meteorological variable of interest (`U_10M` = 10 meter grid eastward wind and `V_10M` = 10 meter grid northward wind). |
| `reference_datetime` | Initialization time of the forecast in **UTC**, provided as either:<br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=0)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT0H"`)|

### Retrieving Data
We now send our requests to the API and retrieve the resulting datasets using the `get_from_ogd()` function.
Each response is returned as an **[xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html)**, which is efficient for handling multi-dimensional data.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing "HTTPError: 403 Client Error: Forbidden for url", you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [ ]:
from earthkit.data import config
config.set("cache-policy", "temporary")

da_U = ogd_api.get_from_ogd(req_U10)
da_V = ogd_api.get_from_ogd(req_V10)

## Data Preprocessing

Once the data has been loaded, continue with the preprocessing of the data if needed. The following preprocessing steps
are described here:
- Calculating horizontal wind speed
- Regridding the data to the regular Swiss grid EPSG:2056

### Calculating Wind Speed
To calculate the horizontal wind speed with the `speed` function from [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/), we need the eastward and northward wind [xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html) `da_U` and `da_V`.

In [ ]:
from meteodatalab.operators import wind

wind_speed = wind.speed(da_U, da_V)
wind_speed

The resulting `xarray.DataArray` has the following dimensions:

- `eps` (ensemble members): 10 members (for ICON-CH1-EPS data)
- `ref_time`: Single reference time (e.g. default date of today midnight)
- `lead_time`: Single lead time (e.g +0 hours)
- `cell`: 1,147,980 spatial grid points

It includes attributes specifying some meta data such as:

- `parameter`: information on the meteorogical parameter, here wind speed
- `vcoord_type`: information of the vertical coordinate type, here heightAboveGround

> &#x1F525; **Expert users:** If you are interested in detailed meta data information, i.e. the GRIB encodings, you can retrieve it from the `metadata` attribute.
Please refer to the [earthkit-data metadata documentation](https://earthkit-data.readthedocs.io/en/latest/examples/metadata.html).

### Regridding to a Regular Grid
To visualize or compare ICON model outputs with other datasets, it is often necessary to regrid the data from its native unstructured grid to a regular latitude/longitude grid.

This can be done using the `regrid` utilities provided by [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/).

We begin by defining a **target grid**, starting with its **spatial extent**, also known as the bounding box. This is specified by the minimum and maximum coordinate values along the x and y axes.

In this example, we use the `regrid.iconremap` function to perform the regridding. Depending on how the target grid's spatial extent is defined, it can fall into one of the following cases:
  - ✅ **Fully overlap** the ICON-CH1/CH2-EPS domain: all target grid points are interpolated.
  - ⚠️ **Partially overlap**: only the overlapping area will contain valid values; other points are filled with `NaN`.
  - ❌ **Lie completely outside** the ICON-CH1/CH2-EPS domain: all target grid values will be `NaN`.

📌 For this example, we define a square area covering Switzerland.


In [ ]:
# Define the target grid extent and resolution
xmin, xmax = 2439000, 2867000 # Meter bounds
ymin, ymax = 1040500, 1334500 # Meter bounds

Next, we define the **number of grid points** (`nx`, `ny`) to control the resolution of the regular grid. We computed `nx` and `ny` to match the resolution of the ICON-CH1-EPS domain (≈ **1 km**).

> 🔢 [How to compute nx, ny for a custom resolution](./computing_nx_ny.md)

In [ ]:
# Compute the number of grid points to match ~2 km resolution
nx, ny = 429, 195             # Number of grid points in x and y

The parameters n<sub>x</sub> and n<sub>y</sub> are passed to `regrid.RegularGrid`, which defines the spatial geometry of the output grid.

In [ ]:
from rasterio.crs import CRS
from meteodatalab.operators import regrid

# Create a regular lat/lon grid using the Swiss grid EPSG:2056
destination = regrid.RegularGrid(
    CRS.from_string("epsg:2056"), nx, ny, xmin, xmax, ymin, ymax
)

Finally, remap the data for each ensemble to the regular Swiss grid EPGS:2056.


In [ ]:
wind_geo_list = []
num_of_eps = len(wind_speed.eps)

for i in range(num_of_eps):
    wind_geo = regrid.iconremap(wind_speed.isel(eps=i), destination)
    wind_geo_list.append(wind_geo)

## Visualizing Wind Speed on a Map
We now visualize the predicted wind speeds for each of the ensemble members on a map. This uses [Matplotlib](https://matplotlib.org/stable/index.html), a wildly used visualization library in Python.

In [ ]:
from cartopy import crs as ccrs
from cartopy import feature as cfeature
import pandas as pd
import matplotlib.pyplot as plt

# Define the Coordinate Reference System (CRS)
crs = ccrs.epsg(2056)

# Create the figure instance
figure = plt.figure(figsize=(14,14))

# Plot the regridded wind speed for each subplot
for idx, wind_geo in enumerate(wind_geo_list):

    # Create a subplot
    ax = figure.add_subplot(4, 3, idx + 1, projection=crs)

    # Remove unnecessary dimensions
    f = wind_geo.squeeze()

    # Plot wind speed
    c = ax.contourf(destination.x, destination.y, f.values, transform=crs)

    # Set subplot title
    eps = wind_geo.coords["eps"].values

    # Add features
    ax.set_title(f"Ensemble {eps}")
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.LAND)
    gl = ax.gridlines(draw_labels=True)

    # Ignore redundant lables
    gl.top_labels = False
    gl.right_labels = False

# Prepare colorbar title and specify location
wind_param = wind_geo_list[0].attrs["parameter"]
legend_label = f"{wind_param['name']} ({wind_param['units']})"
cax = plt.axes((0.41, 0.17, 0.48, 0.03))

# Prepare figure title
ref_time = pd.to_datetime(wind_geo_list[0].coords["ref_time"].values[0]).strftime("%Y-%m-%d %H UTC")
lead_time = wind_geo_list[0].coords["lead_time"].values[0].astype('timedelta64[h]')
title = f"10 Meter Wind Speed | {ref_time} (+{lead_time})"

# Add figure title and colorbar
figure.suptitle(title)
figure.colorbar(c, cax=cax, orientation='horizontal', label=legend_label)
